In [1]:
import os
import netCDF4 as nc
import pandas as pd
import numpy as np

In [2]:
folder_path = 'E:/Work/GiangDay/MohinhhoaMT/K2021/Data_demo'
nc_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.nc4')])

In [3]:
# Biến lưu dữ liệu từng ngày
day_data = []
df = pd.DataFrame()
# Đọc XDim và YDim từ file đầu tiên
first_file = nc.Dataset(os.path.join(folder_path, nc_files[0]), 'r')
XDim = first_file.variables['XDim'][:]
YDim = first_file.variables['YDim'][:]
X, Y = np.meshgrid(XDim, YDim, indexing='ij')

df_base = pd.DataFrame({
    'XDim': X.ravel(),
    'YDim': Y.ravel()
})
first_file.close()

In [4]:
# Tạo DataFrame tổng
df = df_base.copy()

# Đọc dữ liệu NO2 từ từng file
for idx, filename in enumerate(nc_files):
    file_path = os.path.join(folder_path, filename)
    ds = nc.Dataset(file_path, 'r')
    NO2 = ds.variables['ColumnAmountNO2'][:]
    ds.close()
    
    df[f'day{idx+1}'] = NO2.ravel()

# Lưu DataFrame tổng ra file CSV
df.to_csv('NO2_all_days.csv', index=False)

TÍNH TOÁN

In [5]:
import os
import netCDF4 as nc
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('NO2_all_days.csv')

In [7]:
df = df.apply(pd.to_numeric, errors='coerce')
df=df[df>0]
df = df.dropna(thresh=27, axis=1)

In [8]:
value_cols = df.columns[2:]

In [9]:
dflog = df.copy()

In [10]:
dflog[value_cols] = df[value_cols].applymap(lambda x: np.log(x) if isinstance(x, (int, float, np.integer, np.floating)) and x > 0 else np.nan)
df['log_stdev'] = dflog[value_cols].std(axis=1, skipna=True)
df['log_mean'] = dflog[value_cols].mean(axis=1) #Tính AVERAGE (log) dùng trong công thức tính lifetime (t)
df['mean'] = df[value_cols].mean(axis=1) #Tính trung bình bảng dữ liệu ban đầu (để dùng cho tính Emission)

In [11]:
df['time_alive']=(1/(100*(df['log_stdev']/df['log_mean'])))**(1/0.18) #Công thức tính thời gian tồn tại 
df['emission']=(10*1.44*df['mean'])/(df['time_alive']) #Công thức tính M (lượng phát thải)
df.loc[:, ['XDim','YDim','mean','log_stdev','log_mean','time_alive', 'emission']] #in ra các cột có tên trong dấu []

,XDim,YDim,mean,log_stdev,log_mean,time_alive,emission
0,106.375,10.125,3.132968e+15,0.308189,35.636492,2.240960,2.013188e+16
1,106.375,10.375,2.857432e+15,0.329415,35.543979,1.525658,2.697002e+16
2,106.375,10.625,2.776458e+15,0.723827,35.424868,0.018878,2.117823e+18
3,106.375,10.875,2.722472e+15,0.249302,35.511416,7.137918,5.492301e+15
4,106.375,11.125,2.392826e+15,0.259243,35.378979,5.626201,6.124326e+15
5,106.375,11.375,3.963903e+15,0.295349,35.877104,2.946763,1.937048e+16
6,106.625,10.125,3.744416e+15,0.390751,35.791862,0.614113,8.780081e+16
7,106.625,10.375,3.924611e+15,0.298113,35.861096,2.791190,2.024742e+16
8,106.625,10.625,3.547801e+15,0.349640,35.742968,1.130227,4.520186e+16
9,106.625,10.875,3.034512e+15,0.325126,35.590543,1.652874,2.643696e+16
